<a href="https://colab.research.google.com/github/1900690/gradio/blob/main/gradio_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Google Colab上でwebアプリが作れるGradioの使い方](https://clameyes.com/posts/google-colab-web-app-gradio)

[garadioのplaygroundにサンプルがたくさん](https://www.gradio.app/playground)

##※認証がある場合はID;test pass:gradio

In [1]:
#@title gradioライブラリをインストール
#@markdown この共有リンクの有効期限は1週間です。永続的なホスティングとGPUのアップグレードを無料で行うには、作業ディレクトリのターミナルから `gradio deploy` を実行し、Hugging Face Spaces (https://huggingface.co/spaces) にデプロイしてください。
!pip -q install gradio

In [ ]:
#@title 指定したテキストを返すアプリ
import gradio as gr

# 関数を定義
def greet(name):
    return "Hello " + name + "!"

# Webアプリを作成
app = gr.Interface(fn=greet, inputs="text", outputs="text")
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d2db3a0419addaceb0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#@title スライダーで数を定義するアプリ
import gradio as gr

# 関数を定義
def count(number):
    return "数字は " + str(number) + "!"

# Webアプリを作成
app = gr.Interface(fn=count, inputs="slider", outputs="text")
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1d38daa5056d6f920f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[Google ColabでGUI｜gradioを利用したウェブアプリの作り方を解説](https://tech.aru-zakki.com/gradio-on-google-colab/)

In [ ]:
#@title 画像をフィルターに通すアプリ
import numpy as np
import gradio as gr

#セピアフィルターを定義
def sepia(input_img):
    sepia_filter = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ])
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

app = gr.Interface(sepia, gr.Image(), "image")
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://402e2f7432d4e668b6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#@title　入力した値を覚えるアプリ
import gradio as gr

scores = []

def track_score(score):
    scores.append(score)
    top_scores = sorted(scores, reverse=True)[:3]
    return top_scores

app = gr.Interface(
    track_score,
    gr.Number(label="Score"),
    gr.JSON(label="Top Scores")
)
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cd3b70f4d607c1eddb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#アナログメータ針検出アプリをgradio化してみると

In [ ]:
import cv2
import math
import numpy as np
import gradio as gr

def analog(input_img):
        # 指定する中心点（メータの中心）
        cx, cy =(346,304)#横は→に進む、縦は↓に進む
        angle_off_set=-133#pf0の時に０になるように初期値を設定する

        # グレースケール変換 & 二値化
        gray = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        threshold = 100
        _, img_thresh = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)

        # エッジ検出
        edges = cv2.Canny(img_thresh, 50, 200, apertureSize=3)

        # Hough変換で直線を検出（2本の直線を得る）
        minn = None
        for m in range(10, 161, 1):
            lines = cv2.HoughLines(edges, 1, np.pi / 180, m)
            if lines is not None and len(lines) == 2:
                minn = m
                break

        # 直線が2本見つからなかった場合のエラーハンドリング
        if minn is None:
            raise ValueError("直線が2本検出されませんでした。画像のコントラストやエッジ検出のパラメータを調整してください。")

        lines = cv2.HoughLines(edges, 1, np.pi / 180, minn)

        theta_t = []  # 直線の角度
        coefficients = []  # 直線の傾きと切片

        for i in range(len(lines)):
            for rho, theta in lines[i]:
                theta_t.append(theta)

                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a * rho
                y0 = b * rho

                x1 = int(x0 + 1000 * (-b))
                y1 = int(y0 + 1000 * (a))
                x2 = int(x0 - 1000 * (-b))
                y2 = int(y0 - 1000 * (a))
                cv2.line(input_img, (x1, y1), (x2, y2), (0, 0, 255), 2)

                # x2-x1が0の場合のエラーハンドリング
                if x2 - x1 == 0:
                    continue  # 無限傾斜を避ける
                a0 = (y2 - y1) / (x2 - x1)
                b0 = y1 - a0 * x1
                coefficients.append((a0, b0))

        # 直線の数をチェック（2本以上ある場合は最初の2本を使用）
        if len(coefficients) < 2:
            raise gr.Error("十分な直線が検出されませんでした。")

        (a1, b1), (a2, b2) = coefficients[:2]  # 最初の2本を選択

        # 交点を計算
        if a1 == a2:
            raise gr.Error("平行な直線のため交点が計算できません。")

        x_t = (b2 - b1) / (a1 - a2)
        y_t = a1 * x_t + b1

        # 交点と中心点を結ぶ線の角度を計算
        angle_rad = np.arctan2(y_t - cy, x_t - cx)
        angle_deg = np.degrees(angle_rad)
        angle =int(angle_deg) +angle_off_set
        angle=angle*-1  #負の角度を修正
        Kpa=8/27*angle
        if Kpa * 10.197 > 0:
          PH=math.log10(Kpa*10.197)
        elif Kpa == 0 :
          PH=0
        else:
          PH=-1
          raise gr.Error("角度がマイナスになりました")

        # 結果を画像に描画
        img_temp=cv2.circle(input_img, (int(cx), int(cy)), 10, (0, 255, 0), -1)  # 中心点を描画
        img_temp=cv2.circle(img_temp, (int(x_t), int(y_t)), 10, (0, 255, 0), -1)  # 交点を描画
        img_temp=cv2.line(img_temp, (cx, cy), (int(x_t), int(y_t)), (255, 0, 0), 2)  # 中心と交点を結ぶ線
        img_temp=cv2.putText(img_temp, f"{PH:.2f} ph", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        return img_temp

app = gr.Interface(
    analog,
    gr.Image(),
    "image",
     examples=[
         ["https://github.com/1900690/analog-meter-reading/releases/download/analog-meter-samples/000.jpg"],
         ["https://github.com/1900690/analog-meter-reading/releases/download/analog-meter-samples/021.jpg"],
         ["https://github.com/1900690/analog-meter-reading/releases/download/analog-meter-samples/026.jpg"]
     ],
    title="アナログメータ針検出アプリ",
    description="アナログメーターの画像をアップロードしてください。画像からから針の角度を検出します。",
    allow_flagging='never'
    )
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d1d3ce4eeec00673aa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#開空率測定アプリをgradio化してみると

In [ ]:
import cv2
import numpy as np
import gradio as gr


def detect_sky(input_img, Rmax, Rmin, Gmax, Gmin, Bmax, Bmin):
    # Gradio からの画像は RGB 形式
    src = cv2.cvtColor(input_img, cv2.COLOR_RGB2BGR)

    # 各色を取り出し
    r = np.array(src[:, :, 2], dtype=np.int16)
    g = np.array(src[:, :, 1], dtype=np.int16)
    b = np.array(src[:, :, 0], dtype=np.int16)

    # 色フィルタリング
    r_mask = np.where((r >= Rmin) & (r <= Rmax), r, 0)
    g_mask = np.where((g >= Gmin) & (g <= Gmax), g, 0)
    b_mask = np.where((b >= Bmin) & (b <= Bmax), b, 0)

    binde_image = np.zeros_like(src)
    binde_image[:, :, 0] = b_mask
    binde_image[:, :, 1] = g_mask
    binde_image[:, :, 2] = r_mask

    # バイナリマスク作成
    binde_image_binary = (r_mask + g_mask + b_mask)
    binde_image_binary = np.where(binde_image_binary > 0, 255, 0).astype("uint8")

    # 輪郭抽出
    contours, _ = cv2.findContours(binde_image_binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    img_with_line = np.copy(src)
    for i in range(len(contours)):
        if cv2.contourArea(contours[i]) > 0:
            img_with_line = cv2.drawContours(img_with_line, contours, i, (0, 0, 255), 1)

    # 可視化用にRGBに変換
    img_with_line_rgb = cv2.cvtColor(img_with_line, cv2.COLOR_BGR2RGB)

    # 面積マスク作成
    img_with_area = np.zeros_like(src)
    for i in range(len(contours)):
        if cv2.contourArea(contours[i]) > 0:
            img_with_area = cv2.fillPoly(img_with_area, [contours[i][:, 0, :]], (255, 255, 255))

    img_with_area_binary = np.where(np.sum(img_with_area, axis=2) > 0, 255, 0).astype("uint8")
    img_with_area_rgb = np.stack([img_with_area_binary]*3, axis=2)

    # 占有率計算
    all_pixel = img_with_area_binary.size
    sky_pixel = np.sum(img_with_area_binary > 0)
    sky_occupancy_rate = int((sky_pixel / all_pixel) * 100)

    return f"空の面積率: {sky_occupancy_rate}%",img_with_line_rgb, img_with_area_rgb

# Gradioインターフェース
app = gr.Interface(
    fn=detect_sky,
    inputs=[
        gr.Image(type="numpy", label="画像"),
        gr.Slider(minimum=0, maximum=255, value=255, step=1, label="赤上限"),
        gr.Slider(minimum=0, maximum=255, value=200, step=1, label="赤下限"),
        gr.Slider(minimum=0, maximum=255, value=255, step=1, label="緑上限"),
        gr.Slider(minimum=0, maximum=255, value=200, step=1, label="緑下限"),
        gr.Slider(minimum=0, maximum=255, value=255, step=1, label="青上限"),
        gr.Slider(minimum=0, maximum=255, value=200, step=1, label="青下限"),
    ],
    outputs=[
        gr.Label(label="空の面積率"),
        gr.Image(label="空の輪郭表示"),
        gr.Image(label="空の領域マスク")
    ],
    examples=[
    ["https://github.com/1900690/grape-sky-segmentation/releases/download/sample/sample.jpg", 255, 200, 255, 200, 255, 200]
    ],
    allow_flagging='never',
    title="画像から開空率測定アプリ",
    description="画像の空にあたる色をRGBで指定することで空の割合を計算します。",
    article="Copyright (c) 2025 risa iwase  Released under the MIT license  https://opensource.org/licenses/mit-license.php"
)


app.launch()

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:419: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://069578a63bc76b58c1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#ナスLAIアプリをgradio化してみると

In [ ]:
import cv2
import numpy as np
import gradio as gr

# 色の範囲プリセット（背景色の選択肢）
COLOR_RANGES = {
    "葉": {"Rmin": 200, "Rmax": 255, "Gmin": 200, "Gmax": 255, "Bmin": 200, "Bmax": 255},
    "実": {"Rmin": 0, "Rmax": 50, "Gmin": 0, "Gmax": 50, "Bmin": 0, "Bmax": 50},
    "花": {"Rmin": 200, "Rmax": 255, "Gmin": 0, "Gmax": 100, "Bmin": 0, "Bmax": 100},
}

# 空検出メイン関数
def detect_sky(input_img, selected_color, mask_color_hex):
    src = cv2.cvtColor(input_img, cv2.COLOR_RGB2BGR)

    # 背景色の範囲を取得（ラジオボタンで選択された色）
    color_range = COLOR_RANGES[selected_color]
    Rmin, Rmax = color_range["Rmin"], color_range["Rmax"]
    Gmin, Gmax = color_range["Gmin"], color_range["Gmax"]
    Bmin, Bmax = color_range["Bmin"], color_range["Bmax"]

    r = np.array(src[:, :, 2], dtype=np.int16)
    g = np.array(src[:, :, 1], dtype=np.int16)
    b = np.array(src[:, :, 0], dtype=np.int16)

    r_mask = np.where((r >= Rmin) & (r <= Rmax), r, 0)
    g_mask = np.where((g >= Gmin) & (g <= Gmax), g, 0)
    b_mask = np.where((b >= Bmin) & (b <= Bmax), b, 0)

    binde_image_binary = (r_mask + g_mask + b_mask)
    binde_image_binary = np.where(binde_image_binary > 0, 255, 0).astype("uint8")

    contours, _ = cv2.findContours(binde_image_binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    img_with_line = np.copy(src)
    for i in range(len(contours)):
        if cv2.contourArea(contours[i]) > 0:
            img_with_line = cv2.drawContours(img_with_line, contours, i, (0, 0, 255), 1)

    img_with_line_rgb = cv2.cvtColor(img_with_line, cv2.COLOR_BGR2RGB)

    # カラーピッカーで指定された色でマスク塗りつぶし（葉の領域に色をつけたい場合）
    b_col, g_col, r_col = parse_rgb_string(mask_color_hex)
    # 葉以外領域マスクを作成
    mask_leaf = np.zeros(src.shape[:2], dtype=np.uint8)
    for i in range(len(contours)):
        if cv2.contourArea(contours[i]) > 0:
            cv2.fillPoly(mask_leaf, [contours[i][:, 0, :]], 255)
    # マスク反転
    mask_inverse = cv2.bitwise_not(mask_leaf)
    img_with_area = np.zeros_like(src)  # 元画像サイズでマスク色を塗るキャンバス作成
    img_with_area[mask_inverse == 255] = (b_col, g_col, r_col)  # 葉に色を塗る
    img_with_area_rgb = cv2.cvtColor(img_with_area, cv2.COLOR_BGR2RGB)

    # LAIを算出
    reference_pixel_count = 1636516  # 基準画像のピクセル数
    area_per_pixel_of_the_reference_image = 2.170139  # 基準画像の１ピクセル当たりの面積
    img_with_area_binary = np.where(np.sum(img_with_area, axis=2) > 0, 255, 0).astype("uint8")
    all_pixel = img_with_area_binary.size
    reaf_pixcel = np.sum(img_with_area_binary > 0)
    LAI_rate = (reaf_pixcel * area_per_pixel_of_the_reference_image) / 1000000 * (all_pixel / reference_pixel_count)

    return f"葉のLAI: {LAI_rate}", img_with_line_rgb, img_with_area_rgb

# カラーピッカー入力の rgb()/#xxxxxx を BGR タプルに変換する関数
def parse_rgb_string(color_str):
    if color_str.startswith("#"):
        hex_color = color_str.lstrip("#")
        r, g, b = int(hex_color[0:2], 16), int(hex_color[2:4], 16), int(hex_color[4:6], 16)
    else:
        raise ValueError(f"色の形式が無効です: {color_str}")
    return b, g, r  # BGR順

# Gradio インターフェース
app = gr.Interface(
    fn=detect_sky,
    inputs=[
        gr.Image(type="numpy", label="分析する画像"),
        gr.Radio(list(COLOR_RANGES.keys()), label="背景色を選択"),
        gr.ColorPicker(value="#FFFFFF", label="マスクの色（クリックして変更）")
    ],
    outputs=[
        gr.Label(label="葉のLAI"),
        gr.Image(label="葉の輪郭表示"),
        gr.Image(label="葉の領域マスク")
    ],
    examples=[
        ["https://github.com/1900690/eggplant-LAI/releases/download/samle-images/kijun.JPG", "葉", "#ff0000"],
        ["https://github.com/1900690/eggplant-LAI/releases/download/samle-images/200.jpg", "葉", "#ff0000"]
    ],
    allow_flagging='never',
    title="ナスLAI測定アプリ",
    description="画像の背景にあたる色をラジオボタンで選択することでLAIを計算します",
    article="Copyright (c) 2025 risa iwase  Released under the MIT license  https://opensource.org/licenses/mit-license.php"
)

app.launch()


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:425: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d859e82278d59b4343.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import cv2
import numpy as np
import gradio as gr

#検出するものの色スペクトルを指定
COLOR_RANGES = {
    "葉": {"Rmin": 0, "Rmax": 80, "Gmin": 0, "Gmax": 80, "Bmin": 0, "Bmax": 80},
    "実": {"Rmin": 0, "Rmax": 50, "Gmin": 0, "Gmax": 50, "Bmin": 0, "Bmax": 50},
    "枝": {"Rmin": 100, "Rmax": 200, "Gmin": 60, "Gmax": 150, "Bmin": 30, "Bmax": 120},
    "花": {"Rmin": 200, "Rmax": 255, "Gmin": 0, "Gmax": 100, "Bmin": 0, "Bmax": 100},
}

# 固定マスク色（BGR順）を指定
MASK_COLORS = {
    "葉": (0, 0, 255),       # #FF0000
    "実": (238, 26, 240),    # #f01aee
    "枝": (67, 141, 173),    # #AD8D43
    "花": (173, 68, 142),    # #8E44AD
}

# メイン処理
def detect_sky(input_img, selected_color):
    src = cv2.cvtColor(input_img, cv2.COLOR_RGB2BGR)

    # 色スペクトル取得
    cr = COLOR_RANGES[selected_color]
    Rmin, Rmax = cr["Rmin"], cr["Rmax"]
    Gmin, Gmax = cr["Gmin"], cr["Gmax"]
    Bmin, Bmax = cr["Bmin"], cr["Bmax"]

    # 色成分マスク生成
    r = src[:, :, 2]
    g = src[:, :, 1]
    b = src[:, :, 0]

    r_mask = np.where((r >= Rmin) & (r <= Rmax), r, 0)
    g_mask = np.where((g >= Gmin) & (g <= Gmax), g, 0)
    b_mask = np.where((b >= Bmin) & (b <= Bmax), b, 0)

    binary_mask = np.where((r_mask + g_mask + b_mask) > 0, 255, 0).astype(np.uint8)

    # 輪郭抽出
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    img_with_line = cv2.drawContours(src.copy(), contours, -1, (0, 0, 255), 1)
    img_with_line_rgb = cv2.cvtColor(img_with_line, cv2.COLOR_BGR2RGB)

    # 固定色で領域マスク作成
    b_col, g_col, r_col = MASK_COLORS[selected_color]
    mask_obj = np.zeros(src.shape[:2], dtype=np.uint8)
    for cnt in contours:
        if cv2.contourArea(cnt) > 0:
            cv2.fillPoly(mask_obj, [cnt[:, 0, :]], 255)

    #マスクを塗る
    img_with_area = np.zeros_like(src)
    img_with_area[mask_obj == 255] = (b_col, g_col, r_col)
    img_with_area_rgb = cv2.cvtColor(img_with_area, cv2.COLOR_BGR2RGB)

    # 実と花の場合は囲まれる部分の個数を返す
    if selected_color in ["実", "花"]:
        object_count = sum(1 for cnt in contours if cv2.contourArea(cnt) > 0)
        result_text = f"{selected_color}の個数: {object_count}"
    #葉と枝の場合はLAIを返す
    else:
        reference_pixel_count = 1636516# 基準画像のピクセル数（picasosから抽出）
        area_per_pixel_ref = 2.170139# 基準画像の１ピクセル当たりの面積（picasosから抽出）
        img_binary = np.where(np.sum(img_with_area, axis=2) > 0, 255, 0).astype(np.uint8)
        all_pixel = img_binary.size
        target_pixel = np.sum(img_binary > 0)
        LAI_rate = (target_pixel * area_per_pixel_ref) / 1_000_000 * (all_pixel / reference_pixel_count)
        result_text = f"{selected_color}のLAI: {LAI_rate:.3g}"

    return result_text, img_with_line_rgb, img_with_area_rgb


# Gradio UI
app = gr.Interface(
    fn=detect_sky,
    inputs=[
        gr.Image(type="numpy", label="分析する画像"),
        gr.Radio(list(COLOR_RANGES.keys()), label="背景色を選択")
    ],
    outputs=[
        gr.Label(label="計算結果"),
        gr.Image(label="輪郭表示"),
        gr.Image(label="領域マスク")
    ],
    examples=[
        ["https://github.com/1900690/eggplant-LAI/releases/download/samle-images/kijun.JPG", "葉"],
        ["https://github.com/1900690/eggplant-LAI/releases/download/samle-images/200.jpg", "葉"]
    ],
    allow_flagging='never',
    title="ナスLAI測定アプリ",
    description="背景色を選択してLAIまたは個数を計算します（マスク色は固定）",
    article="Copyright (c) 2025 risa iwase  Released under the MIT license  https://opensource.org/licenses/mit-license.php"
)

app.launch()

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:425: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f5c992095bee825818.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [10]:
import cv2
import numpy as np
import gradio as gr
import matplotlib.pyplot as plt
from scipy.signal import find_peaks



# ====== 画像解析ユーティリティ ======
"""引数 hsv は画像をHSV色空間に変換した配列。s_thresh, v_thresh は彩度と明度の下限値（これ以下の色は無視）"""
def auto_leaf_range_from_hist(hsv, s_thresh=40, v_thresh=40):
    """彩度Sと明度Vがそれぞれ閾値より大きいピクセルだけをTrueにするマスク。"""
    mask_sv = (hsv[:, :, 1] > s_thresh) & (hsv[:, :, 2] > v_thresh)
    """上記マスクがTrueのピクセルのHue値（0〜179）だけを取り出す。"""
    hue_values = hsv[:, :, 0][mask_sv]
    """Hue値をヒストグラム化（0〜179度まで、1度ごとにカウント）"""
    hist_h, _ = np.histogram(hue_values, bins=180, range=(0, 180))
    """ヒストグラムから、高さが最大値の30%以上のピークを抽出"""
    peaks, _ = find_peaks(hist_h, height=np.max(hist_h) * 0.3)
    """ピークが見つからなかった場合、デフォルトでHue 25〜90（緑系）を返す。"""
    if len(peaks) == 0:
        return np.array([25, 40, 40]), np.array([90, 255, 255])
    """左側のピークを選択（Hue値が最も小さいピーク）を取得。"""
    main_peak = peaks[np.argmin(peaks)]
    """中心値の前後±15度を葉色範囲とする（範囲外に出ないように制限）。"""
    hue_min = max(0, main_peak - 15)
    hue_max = min(179, main_peak + 15)
    """推定した下限（Hue, S, V）と上限（Hue, 255, 255）を返す。"""
    return np.array([hue_min, 40, 40]), np.array([hue_max, 255, 255])


def generate_hue_histogram(hue_values, h_min, h_max):
    """Hue値と範囲（h_min, h_max）を可視化するためのグラフを作成。"""
    hist_h, _ = np.histogram(hue_values, bins=180, range=(0, 180))
    """Hueの分布を再度ヒストグラム化。"""
    fig, ax = plt.subplots(figsize=(6, 3))
    """緑色の折れ線グラフとして描画。"""
    ax.plot(hist_h, color='g')
    """選択範囲（葉色）を黄色で塗りつぶし表示。"""
    ax.axvspan(h_min, h_max, color='y', alpha=0.3, label='selection range')
    """タイトルと凡例を追加。"""
    ax.set_title('Hue Histogram')
    ax.legend()
    """グラフを画像（RGBA配列）として取得し、メモリ節約のため閉じる。"""
    fig.canvas.draw()
    hist_img = np.array(fig.canvas.renderer.buffer_rgba())
    plt.close(fig)
    """画像として返す。"""
    return hist_img


def process_image(image, h_min, h_max, s_min, v_min):
    """画像を入力し、葉マスク生成・LAI計算・ヒストグラム生成を行う。"""
    hsv = cv2.cvtColor(cv2.cvtColor(image, cv2.COLOR_RGB2BGR), cv2.COLOR_BGR2HSV)
    """Gradioから来る画像はRGBなので、一旦BGRに変換し、さらにHSVへ変換。"""
    # マスク作成
    lower_bound = np.array([h_min, s_min, v_min])
    upper_bound = np.array([h_max, 255, 255])
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    """指定範囲のHSVで二値マスクを作成（葉色だけ白、それ以外黒）。"""
    # ピクセル数計算
    reference_pixel_count = 1636516# 基準画像のピクセル数（picasosから抽出）
    area_per_pixel_ref = 2.170139# 基準画像の１ピクセル当たりの面積（picasosから抽出）
    """葉ピクセル数と全ピクセル数・非葉ピクセル数を計算。"""
    leaf_pixels = int(np.count_nonzero(mask))
    total_pixels = mask.size
    non_leaf_pixels = total_pixels - leaf_pixels
    """LAI（葉面積指数）を計算（㎡単位）。"""
    LAI_rate = (leaf_pixels * area_per_pixel_ref) / 1_000_000 * (total_pixels / reference_pixel_count)
    """結果を文字列として整形。"""
    result_text = f"葉のLAI: {LAI_rate:.3g}"

    """彩度・明度が十分あるピクセルからHueを取り出し、ヒストグラム画像生成。"""
    # ヒストグラム生成
    hue_values = hsv[:, :, 0][(hsv[:, :, 1] > 40) & (hsv[:, :, 2] > 40)]
    hist_img = generate_hue_histogram(hue_values, h_min, h_max)
    """LAI結果（文字列）、マスク画像（RGB変換）、ヒストグラム画像を返す。"""
    return (
        result_text,
        cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB),
        hist_img
    )


def init_sliders(image):
    """画像から自動推定値を取得してスライダー初期化"""
    hsv = cv2.cvtColor(cv2.cvtColor(image, cv2.COLOR_RGB2BGR), cv2.COLOR_BGR2HSV)
    lower, upper = auto_leaf_range_from_hist(hsv)
    return int(lower[0]), int(upper[0]), int(lower[1]), int(lower[2])

# ====== Gradio UI ======

with gr.Blocks(title="葉のLAI計算アプリ") as demo:
    gr.Markdown("## アップロードした画像からLAIを計算します。")

    with gr.Row():
        with gr.Column():
            img_input = gr.Image(type="numpy", label="画像をアップロード")

            # 「解析する」ボタン
            btn_run = gr.Button("解析する", variant="primary")

            # 折りたたみ式の微調整スライダー
            with gr.Accordion("うまく検出できない場合の微調整用", open=False):
                gr.Markdown("HSV色空間で値を調整する（抽出したHueヒストグラムの山をきれいに挟むように色相の上限と下限を調整してください）")
                h_min_slider = gr.Slider(0, 179, step=1, label="色相の下限")
                h_max_slider = gr.Slider(0, 179, step=1, label="色相の上限")
                s_min_slider = gr.Slider(0, 255, step=1, label="彩度の下限")
                v_min_slider = gr.Slider(0, 255, step=1, label="明度の下限")

            gr.Examples(
                examples=[
                    ["https://github.com/1900690/eggplant-LAI/releases/download/samle-images/kijun.JPG"],
                    ["https://github.com/1900690/eggplant-LAI/releases/download/samle-images/200.jpg"]
                ],
                inputs=img_input,
                label="例の画像",
                cache_examples=False
            )

        with gr.Column():
            json_output = gr.Label(label="LAIの計算結果")
            mask_output = gr.Image(type="numpy", label="葉のマスク画像（葉が白、それ以外黒）")
            hist_output = gr.Image(type="numpy", label="抽出したHueヒストグラムの範囲")


    # 画像のアップロード時にHSV色空間を自動推定してスライダーへ反映
    img_input.change(
        fn=init_sliders,
        inputs=img_input,
        outputs=[h_min_slider, h_max_slider, s_min_slider, v_min_slider]
    )

    # 解析実行
    btn_run.click(
        fn=process_image,
        inputs=[img_input, h_min_slider, h_max_slider, s_min_slider, v_min_slider],
        outputs=[json_output, mask_output, hist_output]
    )
    gr.Markdown(
        """
        ---
        <small>Copyright (c) 2025 risa iwase
        Released under the MIT license
        <a href="https://opensource.org/licenses/mit-license.php" target="_blank">https://opensource.org/licenses/mit-license.php</a></small>
        """
    )

if __name__ == "__main__":
    demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a376d25a3b129dac11.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#画像からカラーヒストグラを作成するアプリをgradio化してみると

In [ ]:
import gradio as gr
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import os
from uuid import uuid4

def calculate_histograms(image_rgb):
    """RGB画像から各チャンネルのヒストグラムを計算する"""
    colors = ['Red', 'Green', 'Blue']
    hist_data = {
        col: cv2.calcHist([image_rgb], [i], None, [256], [0, 256]).flatten()
        for i, col in enumerate(colors)
    }
    return hist_data

def plot_histogram(hist_data):
    """ヒストグラムデータをプロットしてmatplotlibのFigureを返す"""
    fig = plt.figure(figsize=(6, 4))
    for col in hist_data:
        plt.plot(hist_data[col], color=col.lower())
    plt.title("RGB Color Histogram")
    plt.xlabel("Pixel Value")
    plt.ylabel("Frequency")
    return fig

def figure_to_image(fig):
    """matplotlibのFigureをPIL画像に変換"""
    fig.canvas.draw()
    w, h = fig.canvas.get_width_height()
    img_np = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8).reshape((h, w, 4))
    plt.close(fig)
    return Image.fromarray(cv2.cvtColor(img_np, cv2.COLOR_RGBA2RGB))

def save_histogram_csv(hist_data):
    """ヒストグラムデータをCSVとして保存し、ファイルパスを返す"""
    df = pd.DataFrame(hist_data)
    df.index.name = "Pixel Value"
    csv_path = f"/tmp/histogram_{uuid4().hex}.csv"
    df.to_csv(csv_path)
    return csv_path

def create_histogram_and_csv(image):
    """Gradioから呼び出されるメイン処理"""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    hist_data = calculate_histograms(image_rgb)
    fig = plot_histogram(hist_data)
    histogram_img = figure_to_image(fig)
    csv_path = save_histogram_csv(hist_data)
    return histogram_img, csv_path

# Gradio インターフェース
demo = gr.Interface(
    fn=create_histogram_and_csv,
    inputs=gr.Image(type="numpy", label="画像をアップロードする"),
    outputs=[
        gr.Image(type="pil", label="RGB ヒストグラム"),
        gr.File(label="ヒストグラムのCSVをダウンロード")
    ],
    examples=[
        ["https://github.com/1900690/eggplant-LAI/releases/download/samle-images/kijun.JPG"],
        ["https://github.com/1900690/eggplant-LAI/releases/download/samle-images/200.jpg"]
    ],
    title="RGBヒストグラム生成ツール",
    description="画像をアップロードしてそのRGBヒストグラムを表示し、またヒストグラムの値をCSVファイルとしてダウンロードできます。",
    article="Copyright (c) 2025 risa iwase  Released under the MIT license  https://opensource.org/licenses/mit-license.php"
)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://60524e8d408a298b7f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#画像とそのマスク画像から重なりを抽出するアプリをgradio化してみると

In [ ]:
import gradio as gr
import numpy as np
import cv2

def extract_mask_transparent(image, mask):
    if image is None or mask is None:
        return None

    # 画像が3チャンネル(RGB)であることを保証
    if image.shape[2] == 4:
        image = image[:, :, :3]

    # マスク画像をグレースケール化
    if len(mask.shape) == 3:
        mask_gray = cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY)
    else:
        mask_gray = mask

    # マスクの白い部分(=255)を不透明、それ以外を透明にしたアルファチャンネルを作成
    _, mask_bin = cv2.threshold(mask_gray, 127, 255, cv2.THRESH_BINARY)
    alpha = mask_bin.astype(np.uint8)

    # RGBA画像を作成
    rgba = cv2.cvtColor(image, cv2.COLOR_RGB2RGBA)
    rgba[:, :, 3] = alpha

    return rgba

iface = gr.Interface(
    fn=extract_mask_transparent,
    inputs=[
        gr.Image(type="numpy", label="元画像をアップロード"),
        gr.Image(type="numpy", label="マスク画像をアップロード")
    ],
    outputs=gr.Image(type="numpy", label="マスク重なり部分"),
    examples=[
        ["https://github.com/1900690/eggplant-LAI/releases/download/samle-images/200.jpg","https://github.com/1900690/eggplant-LAI/releases/download/samle-images/200mask.jpg"],
    ],
    title="画像とマスクの重なり部分抽出ツール",
    description="画像とマスクの重なり部分を抽出してダウンロードできます",
    article="Copyright (c) 2025 risa iwase  Released under the MIT license  https://opensource.org/licenses/mit-license.php",
    allow_flagging="never"
)

if __name__ == "__main__":
    iface.launch()